# Analyze Project Sprint 

In [1]:
from crewai import Agent, Task, Crew

In [9]:
import os 
import yaml
import json
from dotenv import load_dotenv
load_dotenv('../../.env')

model= 'azure/gpt-4o-mini' #os.environ.get('model')
print(model)

sprint_no = '2451'

azure/gpt-4o-mini


In [10]:
# Define file paths for YAML configurations
files = {
    'agents': 'analyze_sprint/agents.yaml',
    'tasks': 'analyze_sprint/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Creating Custom Tools

In [11]:
from crewai.tools import BaseTool
import requests

class SprintDataFetcherTool(BaseTool):
    name: str = "Azure DevOps Data Fetcher"
    description: str = "Fetches data for user stories and bugs from an Azure DevOps project"
    
    def _run(self) -> dict:
        """
        Fetch data from Azure DevOps Project for all stories and bugs that were worked upon in the sprint
        """
        file = 'analyze_sprint/'+sprint_no+'.txt'
        #open and read the file after the overwriting:
        f = open(file, "r")
        data = f.read()
        f.close()
        return data

## Result

In [12]:
# Creating Agents
data_collection_agent = Agent(
  config=agents_config['data_collection_agent'],
  tools=[SprintDataFetcherTool()]
)

analysis_agent = Agent(
  config=agents_config['analysis_agent']
)

# Creating Tasks
data_collection = Task(
  config=tasks_config['data_collection'],
  agent=data_collection_agent
)

data_analysis = Task(
  config=tasks_config['data_analysis'],
  agent=analysis_agent
)

report_generation = Task(
  config=tasks_config['report_generation'],
  agent=analysis_agent,
)

# Creating Crew
crew = Crew(
  agents=[
    data_collection_agent,
    analysis_agent
  ],
  tasks=[
    data_collection,
    data_analysis,
    report_generation
  ],
  verbose=True
)

Overriding of current TracerProvider is not allowed


In [13]:
# Kick off the crew and execute the process
result = crew.kickoff()

# Agent: Data Collection Specialist\n
## Task: Create an initial understanding of the project, its main features and the team working on it. Use the Sprint Data Fetcher Tool to gather data from Azure DevOps.\n


# Agent: Data Collection Specialist\n
## Thought: Thought: I must begin by collecting the data from Azure DevOps to create a detailed report on the project. However, the tool provided "Azure DevOps Data Fetcher" is designed to fetch data for user stories and bugs, and not necessarily for a broader project overview including team information or main features. To comply with the instructions, I will use the tool provided to gather detailed information on user stories and bugs.
## Using tool: Azure DevOps Data Fetcher
## Tool Input: 
"{}"
## Tool Output: 
Type: User Story<br/>Title: New Order Overview - Super User reusable Component to be used in both Order Overview Compoenents(Spill)<br/>State: 5 Accepted<br/>CreatedBy: partner.Shreelakshmi.Venugopal@philips.com;  CreatedDate: 20

In [14]:
from IPython.display import display, Markdown

output = result.raw

# Display the formatted output as Markdown
display(Markdown(output))

# Sprint Report

## Sprint Overview
The sprint has been marked by satisfactory progression with the smooth closure of a user story and a bug. There has been a commendable emphasis on collaboration, security, and quality assurance during this sprint.

## Task Summary
### User Story:
- **Title:** New Order Overview - Super User Reusable Component
- **State:** Accepted
- **Created By:** partner.Shreelakshmi.Venugopal
- **Assigned To:** partner.Sreekanth.M
- **Resolved By:** partner.Chandrashekhar.Badgujar
- **Closed By:** partner.Shreelakshmi.Venugopal
- **Story Points:** 2.0
- **Tags:** CP2025Q1; SIT Deployed

### Bug:
- **Title:** ST || Vulnerable version of the library 'DOMPurify' found
- **State:** Accepted
- **Created By:** partner.Jayesh.Thakare
- **Assigned To:** partner.Jayesh.Thakare
- **Resolved By:** partner.raju.sharma
- **Closed By:** partner.Jayesh.Thakare
- **Story Points:** N/A
- **Tags:** Mindtree_ST; Security; SFFT

## Identified Issues and Blockers
There have been no significant blockers or delays, but potential initial prioritization delays were noted in the bug resolution phase due to the absence of an activation date.

## Progress and Delays
The progress of user stories and bug resolution has stayed within acceptable time frames. However, the non-immediate activation of the bug and the month-long gap between resolution and acceptance of the user story suggest areas for improvement.

## Team Performance Overview
The team exhibited high levels of efficiency, especially given the 2.0 story point estimation for the user story and the successful deployment after system integration testing. The security bug was handled with clear ownership and resolved effectively within two months.

## Action Items and Recommendations
- **Improve Activation Time:** Investigate the activation process for bugs and issues to speed up response time.
- **Streamline Acceptance Process:** Review the acceptance process to minimize delays after issue resolution.
- **Encourage Proactive Communication:** Maintain the high level of collaboration seen in this sprint and continue using specialized tag categories for more accessible tracking and reporting.

## Additionals
- **Quality Focus:** 'SIT Deployed' and 'SFFT' tags should continue to be a priority to ensure the high quality of the codebase.
- **Continued Monitoring:** Keep an eye on the lead times between resolution and final acceptance of user stories and bugs for a more streamlined process.

This report highlights areas of strength and potential improvement, contributing to ongoing project success and delivery excellence.

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [42]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0008


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,5311,3704,0,1607,4


In [16]:
crewai log-tasks-outputs

'crewai' is not recognized as an internal or external command,
operable program or batch file.
